In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
sys.path.append(str(Path().resolve().parent.parent))

import numpy as np
import transforms.transforms as trf
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
import math
from losses import CustomCrossEntropyLoss
import torchvision.models as models

import json

In [3]:
r="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L595310_r.npy"
g="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L595310_g.npy"
b="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L595310_b.npy"
m = "/miniscratch/akera/ecosystem-embedding/data/sentinel2/L595310.json"
s = "/miniscratch/tengmeli/ecosystem-embedding/ebird_data_june/L595310.json"

r1="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L999786_r.npy"
g1="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L999786_g.npy"
b1="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L999786_b.npy"
m1="/miniscratch/akera/ecosystem-embedding/data/sentinel2/L999786.json"
s1 = "/miniscratch/tengmeli/ecosystem-embedding/ebird_data_june/L595310.json"

df_paths = {'r': [r, r1], 'g': [g,g1], 'b':[b, b1], "metadata":[m,m1], "species":[s,s1]}

df_paths = pd.DataFrame(df_paths)

df = df_paths
total_images = len(df_paths)
#maybe have to write get_transforms funciton ??
#transform = transforms
bands = ['r', 'g', 'b']
mode = "train"

In [4]:
np.load(r1)

array([[[ 639,  606,  683, ...,  873,  683,  677],
        [ 679,  641,  777, ...,  651,  642,  666],
        [ 720,  670,  688, ...,  652,  734,  737],
        ...,
        [2066, 1552, 1384, ..., 1072,  916, 1310],
        [1132, 1548, 1676, ...,  916, 1042, 1196],
        [1360, 1948, 1670, ...,  601, 1158, 1152]]], dtype=uint16)

In [19]:
from dataset.dataloader import EbirdVisionDataset
from transforms.transforms import RandomCrop, RandomHorizontalFlip, Normalize, RandomGaussianNoise
from torchvision import transforms as trsfs

bands = ["r","g", "b"]
dataset = EbirdVisionDataset(df,bands, trsfs.Compose([RandomCrop((256,256), center=True), RandomHorizontalFlip(), RandomGaussianNoise(), Normalize()]))

In [20]:
from dataset.utils import load_file
from dataset.dataloader import get_path
import torch

In [21]:
#load_file(Path("/miniscratch/tengmeli/ecosystem-embedding/ebird_data_june/L595310.json"))

In [22]:
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1, pin_memory=True, drop_last = True)

device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 2

In [23]:
next(iter(train_dataloader))

{'sat': tensor([[[[[0.0541, 0.0616, 0.0640,  ..., 0.0623, 0.0652, 0.0574],
            [0.0529, 0.0543, 0.0539,  ..., 0.0559, 0.0552, 0.0510],
            [0.0564, 0.0547, 0.0496,  ..., 0.0491, 0.0500, 0.0428],
            ...,
            [0.0668, 0.0636, 0.0922,  ..., 0.1514, 0.1622, 0.1837],
            [0.0514, 0.0504, 0.1199,  ..., 0.1603, 0.1722, 0.1697],
            [0.0494, 0.0639, 0.1101,  ..., 0.1608, 0.1622, 0.1637]],
 
           [[0.0741, 0.0828, 0.0869,  ..., 0.0806, 0.0820, 0.0775],
            [0.0696, 0.0781, 0.0851,  ..., 0.0771, 0.0741, 0.0675],
            [0.0720, 0.0729, 0.0777,  ..., 0.0681, 0.0673, 0.0581],
            ...,
            [0.0903, 0.0786, 0.1171,  ..., 0.1458, 0.1590, 0.1775],
            [0.0704, 0.0565, 0.1480,  ..., 0.1650, 0.1626, 0.1826],
            [0.0694, 0.0611, 0.1337,  ..., 0.1669, 0.1609, 0.1665]],
 
           [[0.0435, 0.0521, 0.0507,  ..., 0.0544, 0.0527, 0.0489],
            [0.0397, 0.0414, 0.0515,  ..., 0.0470, 0.0472, 0.0428],
 

In [10]:
criterion = CustomCrossEntropyLoss()#BCEWithLogitsLoss()

In [11]:
model = models.resnet18(pretrained = True)
model.fc = nn.Linear(512, 684) 
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
num_epochs = 20

In [13]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
m= nn.Sigmoid()
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    
    if mode == 'train':
        model.train()
    else:
        model.eval()

    for elem in train_dataloader:
        
        optimizer.zero_grad()
        
        inputs = elem["sat"].squeeze(1).to(device)
        labels = elem["target"].to(device)
        outputs = model(inputs)
        
        loss = criterion(m(outputs), labels)
        #print(loss)
        loss.backward()
        optimizer.step()
        
        
        print(nn.MSELoss()(m(outputs), labels))
       # running_loss += loss
   # print(running_loss)
    #running_loss = 0.0